In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing 
import warnings
import statsmodels.api as sm
!pip install GEKKO
!pip install ortools

from gekko import GEKKO

In [2]:
Download=pd.read_csv("DKSalaries-5.csv")

In [3]:
Download["KingCost"]=Download["Salary"]*1.5

In [4]:
Download

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
0,QB,Justin Herbert (19490421),Justin Herbert,19490421,CPT,16800,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,25200.0
1,QB,Derek Carr (19490422),Derek Carr,19490422,CPT,16200,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,24300.0
2,WR,Mike Williams (19490423),Mike Williams,19490423,CPT,15300,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,22950.0
3,RB,Austin Ekeler (19490424),Austin Ekeler,19490424,CPT,14400,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,21600.0
4,TE,Darren Waller (19490425),Darren Waller,19490425,CPT,13800,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,20700.0
...,...,...,...,...,...,...,...,...,...,...
95,TE,Tre' McKitty (19490516),Tre' McKitty,19490516,FLEX,200,LV@LAC 10/04/2021 08:15PM ET,LAC,0.00,300.0
96,RB,Joshua Kelley (19490517),Joshua Kelley,19490517,FLEX,200,LV@LAC 10/04/2021 08:15PM ET,LAC,0.00,300.0
97,WR,Maurice Ffrench (19490518),Maurice Ffrench,19490518,FLEX,200,LV@LAC 10/04/2021 08:15PM ET,LAC,0.00,300.0
98,WR,Dillon Stoner (19490519),Dillon Stoner,19490519,FLEX,200,LV@LAC 10/04/2021 08:15PM ET,LV,0.00,300.0


In [5]:
Download=Download.loc[((Download['Roster Position'] != "CPT"))]

In [6]:

Download.index = np.arange(1, len(Download)+1)
Download

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Justin Herbert (19490471),Justin Herbert,19490471,FLEX,11200,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,16800.0
2,QB,Derek Carr (19490472),Derek Carr,19490472,FLEX,10800,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,16200.0
3,WR,Mike Williams (19490473),Mike Williams,19490473,FLEX,10200,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,15300.0
4,RB,Austin Ekeler (19490474),Austin Ekeler,19490474,FLEX,9600,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,14400.0
5,TE,Darren Waller (19490475),Darren Waller,19490475,FLEX,9200,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,13800.0
6,WR,Keenan Allen (19490476),Keenan Allen,19490476,FLEX,8800,LV@LAC 10/04/2021 08:15PM ET,LAC,19.60,13200.0
7,RB,Josh Jacobs (19490477),Josh Jacobs,19490477,FLEX,7600,LV@LAC 10/04/2021 08:15PM ET,LV,17.00,11400.0
8,RB,Peyton Barber (19490478),Peyton Barber,19490478,FLEX,6800,LV@LAC 10/04/2021 08:15PM ET,LV,14.70,10200.0
9,WR,Henry Ruggs III (19490479),Henry Ruggs III,19490479,FLEX,6400,LV@LAC 10/04/2021 08:15PM ET,LV,14.87,9600.0
10,QB,Kyle Sloter (19490480),Kyle Sloter,19490480,FLEX,6000,LV@LAC 10/04/2021 08:15PM ET,LV,0.00,9000.0


In [7]:
Modified=Download

In [8]:

Modified = Modified.drop(50)
Modified = Modified.drop(49)
Modified = Modified.drop(48)
Modified = Modified.drop(47)
Modified = Modified.drop(46)
Modified = Modified.drop(45)
Modified = Modified.drop(44)
Modified = Modified.drop(43)
Modified = Modified.drop(42)
Modified = Modified.drop(41)
Modified = Modified.drop(37)
Modified = Modified.drop(39)
Modified = Modified.drop(38)
Modified = Modified.drop(35)
Modified = Modified.drop(34)
Modified = Modified.drop(33)
Modified = Modified.drop(32)
Modified = Modified.drop(30)
Modified = Modified.drop(10)
Modified = Modified.drop(11)
Modified = Modified.drop(12)
Modified = Modified.drop(13)
Modified = Modified.drop(14)
Modified = Modified.drop(24)
Modified = Modified.drop(26)
Modified = Modified.drop(27)
Modified = Modified.drop(19)

In [9]:
Modified

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Justin Herbert (19490471),Justin Herbert,19490471,FLEX,11200,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,16800.0
2,QB,Derek Carr (19490472),Derek Carr,19490472,FLEX,10800,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,16200.0
3,WR,Mike Williams (19490473),Mike Williams,19490473,FLEX,10200,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,15300.0
4,RB,Austin Ekeler (19490474),Austin Ekeler,19490474,FLEX,9600,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,14400.0
5,TE,Darren Waller (19490475),Darren Waller,19490475,FLEX,9200,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,13800.0
6,WR,Keenan Allen (19490476),Keenan Allen,19490476,FLEX,8800,LV@LAC 10/04/2021 08:15PM ET,LAC,19.60,13200.0
7,RB,Josh Jacobs (19490477),Josh Jacobs,19490477,FLEX,7600,LV@LAC 10/04/2021 08:15PM ET,LV,17.00,11400.0
8,RB,Peyton Barber (19490478),Peyton Barber,19490478,FLEX,6800,LV@LAC 10/04/2021 08:15PM ET,LV,14.70,10200.0
9,WR,Henry Ruggs III (19490479),Henry Ruggs III,19490479,FLEX,6400,LV@LAC 10/04/2021 08:15PM ET,LV,14.87,9600.0
15,WR,Hunter Renfrow (19490485),Hunter Renfrow,19490485,FLEX,5800,LV@LAC 10/04/2021 08:15PM ET,LV,14.13,8700.0


In [10]:

Modified.index = np.arange(1, len(Modified)+1)

In [11]:
Modified



,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Justin Herbert (19490471),Justin Herbert,19490471,FLEX,11200,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,16800.0
2,QB,Derek Carr (19490472),Derek Carr,19490472,FLEX,10800,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,16200.0
3,WR,Mike Williams (19490473),Mike Williams,19490473,FLEX,10200,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,15300.0
4,RB,Austin Ekeler (19490474),Austin Ekeler,19490474,FLEX,9600,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,14400.0
5,TE,Darren Waller (19490475),Darren Waller,19490475,FLEX,9200,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,13800.0
6,WR,Keenan Allen (19490476),Keenan Allen,19490476,FLEX,8800,LV@LAC 10/04/2021 08:15PM ET,LAC,19.60,13200.0
7,RB,Josh Jacobs (19490477),Josh Jacobs,19490477,FLEX,7600,LV@LAC 10/04/2021 08:15PM ET,LV,17.00,11400.0
8,RB,Peyton Barber (19490478),Peyton Barber,19490478,FLEX,6800,LV@LAC 10/04/2021 08:15PM ET,LV,14.70,10200.0
9,WR,Henry Ruggs III (19490479),Henry Ruggs III,19490479,FLEX,6400,LV@LAC 10/04/2021 08:15PM ET,LV,14.87,9600.0
10,WR,Hunter Renfrow (19490485),Hunter Renfrow,19490485,FLEX,5800,LV@LAC 10/04/2021 08:15PM ET,LV,14.13,8700.0


In [12]:
Code_list=['x1x2','x3x4','x5x6','x7x8','x9x10',
           'x11x12','x13x14','x15x16','x17x18',
           'x19x20','x21x22','x23x24','x25x26',
           'x27x28','x29x30','x31x32','x33x34',
           'x35x36','x37x38','x39x40','x41x42',
           'x43x44', 'x45x46']

In [13]:
Modified.index = Code_list

In [14]:
Modified

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
x1x2,QB,Justin Herbert (19490471),Justin Herbert,19490471,FLEX,11200,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,16800.0
x3x4,QB,Derek Carr (19490472),Derek Carr,19490472,FLEX,10800,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,16200.0
x5x6,WR,Mike Williams (19490473),Mike Williams,19490473,FLEX,10200,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,15300.0
x7x8,RB,Austin Ekeler (19490474),Austin Ekeler,19490474,FLEX,9600,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,14400.0
x9x10,TE,Darren Waller (19490475),Darren Waller,19490475,FLEX,9200,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,13800.0
x11x12,WR,Keenan Allen (19490476),Keenan Allen,19490476,FLEX,8800,LV@LAC 10/04/2021 08:15PM ET,LAC,19.60,13200.0
x13x14,RB,Josh Jacobs (19490477),Josh Jacobs,19490477,FLEX,7600,LV@LAC 10/04/2021 08:15PM ET,LV,17.00,11400.0
x15x16,RB,Peyton Barber (19490478),Peyton Barber,19490478,FLEX,6800,LV@LAC 10/04/2021 08:15PM ET,LV,14.70,10200.0
x17x18,WR,Henry Ruggs III (19490479),Henry Ruggs III,19490479,FLEX,6400,LV@LAC 10/04/2021 08:15PM ET,LV,14.87,9600.0
x19x20,WR,Hunter Renfrow (19490485),Hunter Renfrow,19490485,FLEX,5800,LV@LAC 10/04/2021 08:15PM ET,LV,14.13,8700.0


In [15]:
Download=Modified

In [16]:
Download

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
x1x2,QB,Justin Herbert (19490471),Justin Herbert,19490471,FLEX,11200,LV@LAC 10/04/2021 08:15PM ET,LAC,22.98,16800.0
x3x4,QB,Derek Carr (19490472),Derek Carr,19490472,FLEX,10800,LV@LAC 10/04/2021 08:15PM ET,LV,26.81,16200.0
x5x6,WR,Mike Williams (19490473),Mike Williams,19490473,FLEX,10200,LV@LAC 10/04/2021 08:15PM ET,LAC,26.83,15300.0
x7x8,RB,Austin Ekeler (19490474),Austin Ekeler,19490474,FLEX,9600,LV@LAC 10/04/2021 08:15PM ET,LAC,18.97,14400.0
x9x10,TE,Darren Waller (19490475),Darren Waller,19490475,FLEX,9200,LV@LAC 10/04/2021 08:15PM ET,LV,17.13,13800.0
x11x12,WR,Keenan Allen (19490476),Keenan Allen,19490476,FLEX,8800,LV@LAC 10/04/2021 08:15PM ET,LAC,19.60,13200.0
x13x14,RB,Josh Jacobs (19490477),Josh Jacobs,19490477,FLEX,7600,LV@LAC 10/04/2021 08:15PM ET,LV,17.00,11400.0
x15x16,RB,Peyton Barber (19490478),Peyton Barber,19490478,FLEX,6800,LV@LAC 10/04/2021 08:15PM ET,LV,14.70,10200.0
x17x18,WR,Henry Ruggs III (19490479),Henry Ruggs III,19490479,FLEX,6400,LV@LAC 10/04/2021 08:15PM ET,LV,14.87,9600.0
x19x20,WR,Hunter Renfrow (19490485),Hunter Renfrow,19490485,FLEX,5800,LV@LAC 10/04/2021 08:15PM ET,LV,14.13,8700.0


In [17]:
m = GEKKO() # create GEKKO model
# create binary variables
x1 = m.Var(integer=True,lb=0,ub=1) 
x2 = m.Var(integer=True,lb=0,ub=1)
x3 = m.Var(integer=True,lb=0,ub=1)
x4 = m.Var(integer=True,lb=0,ub=1)
x5 = m.Var(integer=True,lb=0,ub=1)
x6 = m.Var(integer=True,lb=0,ub=1)
x7 = m.Var(integer=True,lb=0,ub=1)
x8 = m.Var(integer=True,lb=0,ub=1)
x9 = m.Var(integer=True,lb=0,ub=1)
x10 = m.Var(integer=True,lb=0,ub=1)
x11 = m.Var(integer=True,lb=0,ub=1)
x12 = m.Var(integer=True,lb=0,ub=1)
x13 = m.Var(integer=True,lb=0,ub=1)
x14 = m.Var(integer=True,lb=0,ub=1)
x15 = m.Var(integer=True,lb=0,ub=1)
x16 = m.Var(integer=True,lb=0,ub=1)
x17 = m.Var(integer=True,lb=0,ub=1)
x18 = m.Var(integer=True,lb=0,ub=1)
x19 = m.Var(integer=True,lb=0,ub=1)
x20 = m.Var(integer=True,lb=0,ub=1)
x21 = m.Var(integer=True,lb=0,ub=1)
x22 = m.Var(integer=True,lb=0,ub=1)
x23 = m.Var(integer=True,lb=0,ub=1)
x24 = m.Var(integer=True,lb=0,ub=1)
x25 = m.Var(integer=True,lb=0,ub=1)
x26 = m.Var(integer=True,lb=0,ub=1)
x27 = m.Var(integer=True,lb=0,ub=1)
x28 = m.Var(integer=True,lb=0,ub=1)
x29 = m.Var(integer=True,lb=0,ub=1) 
x30 = m.Var(integer=True,lb=0,ub=1)
x31 = m.Var(integer=True,lb=0,ub=1)
x32 = m.Var(integer=True,lb=0,ub=1)
x33 = m.Var(integer=True,lb=0,ub=1)
x34 = m.Var(integer=True,lb=0,ub=1)
x35 = m.Var(integer=True,lb=0,ub=1)
x36 = m.Var(integer=True,lb=0,ub=1)
x37 = m.Var(integer=True,lb=0,ub=1)
x38 = m.Var(integer=True,lb=0,ub=1)
x39 = m.Var(integer=True,lb=0,ub=1)
x40 = m.Var(integer=True,lb=0,ub=1)
x41 = m.Var(integer=True,lb=0,ub=1)
x42 = m.Var(integer=True,lb=0,ub=1)
x43 = m.Var(integer=True,lb=0,ub=1)
x44 = m.Var(integer=True,lb=0,ub=1)
x45 = m.Var(integer=True,lb=0,ub=1)
x46 = m.Var(integer=True,lb=0,ub=1)

In [19]:

m.Equation(x2+x4+x6+x8+x10+x12+x14+x16+x18+x20+x22+x24+x26+x28+
           x30+x32+x34+x36+x38+x40+x42+x44+x46==1)

m.Equation(x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11+x12+x13+x14+
           x15+x16+x17+x18+x19+x20+x21+x22+x23+x24+x25+x26+x27+x28+
           x29+x30+x31+x32+x33+x34+x35+x36+x37+x38+x39+x40+x41+x42+x43+x44+x45+x46== 6)



m.Equation(x1+x2<=1)
m.Equation(x3+x4<=1)
m.Equation(x5+x6<=1)
m.Equation(x7+x8<=1)
m.Equation(x9+x10<=1)
m.Equation(x11+x12<=1)
m.Equation(x13+x14<=1)
m.Equation(x15+x16<=1)
m.Equation(x17+x18<=1)
m.Equation(x19+x20<=1)
m.Equation(x21+x22<=1)
m.Equation(x23+x24<=1)
m.Equation(x25+x26<=1)
m.Equation(x27+x28<=1)
m.Equation(x29+x30<=1)
m.Equation(x31+x32<=1)
m.Equation(x33+x34<=1)
m.Equation(x35+x36<=1)
m.Equation(x37+x38<=1)
m.Equation(x39+x40<=1)
m.Equation(x41+x42<=1)
m.Equation(x43+x44<=1)
m.Equation(x45+x46<=1)

m.Equation((x1*Download.iloc[0]['Salary'])+(x2*Download.iloc[0]['Salary']*1.5)+
            (x3*Download.iloc[1]['Salary'])+(x4*Download.iloc[1]['Salary']*1.5)+
            (x5*Download.iloc[2]['Salary'])+(x6*Download.iloc[2]['Salary']*1.5)+
            (x7*Download.iloc[3]['Salary'])+(x8*Download.iloc[3]['Salary']*1.5)+
            (x9*Download.iloc[4]['Salary'])+(x10*Download.iloc[4]['Salary']*1.5)+
            (x11*Download.iloc[5]['Salary'])+(x12*Download.iloc[5]['Salary']*1.5)+
            (x13*Download.iloc[6]['Salary'])+(x14*Download.iloc[6]['Salary']*1.5)+
            (x15*Download.iloc[7]['Salary'])+(x16*Download.iloc[7]['Salary']*1.5)+
            (x17*Download.iloc[8]['Salary'])+(x18*Download.iloc[8]['Salary']*1.5)+
            (x19*Download.iloc[9]['Salary'])+(x20*Download.iloc[9]['Salary']*1.5)+
            (x21*Download.iloc[10]['Salary'])+(x22*Download.iloc[10]['Salary']*1.5)+
            (x23*Download.iloc[11]['Salary'])+(x24*Download.iloc[11]['Salary']*1.5)+
            (x25*Download.iloc[12]['Salary'])+(x26*Download.iloc[12]['Salary']*1.5)+
            (x27*Download.iloc[13]['Salary'])+(x28*Download.iloc[13]['Salary']*1.5)+
            (x29*Download.iloc[14]['Salary'])+(x30*Download.iloc[14]['Salary']*1.5)+
            (x31*Download.iloc[15]['Salary'])+(x32*Download.iloc[15]['Salary']*1.5)+
            (x33*Download.iloc[16]['Salary'])+(x34*Download.iloc[16]['Salary']*1.5)+
            (x35*Download.iloc[17]['Salary'])+(x36*Download.iloc[17]['Salary']*1.5)+
            (x37*Download.iloc[18]['Salary'])+(x38*Download.iloc[18]['Salary']*1.5)+
            (x39*Download.iloc[19]['Salary'])+(x40*Download.iloc[19]['Salary']*1.5)+
            (x41*Download.iloc[20]['Salary'])+(x42*Download.iloc[20]['Salary']*1.5)+
            (x43*Download.iloc[21]['Salary'])+(x44*Download.iloc[21]['Salary']*1.5)+
            (x45*Download.iloc[22]['Salary'])+(x46*Download.iloc[22]['Salary']*1.5) <=50000)


m.Maximize(((x1*Download.iloc[0]['AvgPointsPerGame'])+(x2*Download.iloc[0]['AvgPointsPerGame']*1.5)+
        (x3*Download.iloc[1]['AvgPointsPerGame'])+(x4*Download.iloc[1]['AvgPointsPerGame']*1.5)+
        (x5*Download.iloc[2]['AvgPointsPerGame'])+(x6*Download.iloc[2]['AvgPointsPerGame']*1.5)+
        (x7*Download.iloc[3]['AvgPointsPerGame'])+(x8*Download.iloc[3]['AvgPointsPerGame']*1.5)+
        (x9*Download.iloc[4]['AvgPointsPerGame'])+(x10*Download.iloc[4]['AvgPointsPerGame']*1.5)+
        (x11*Download.iloc[5]['AvgPointsPerGame'])+(x12*Download.iloc[5]['AvgPointsPerGame']*1.5)+
        (x13*Download.iloc[6]['AvgPointsPerGame'])+(x14*Download.iloc[6]['AvgPointsPerGame']*1.5)+
        (x15*Download.iloc[7]['AvgPointsPerGame'])+(x16*Download.iloc[7]['AvgPointsPerGame']*1.5)+
        (x17*Download.iloc[8]['AvgPointsPerGame'])+(x18*Download.iloc[8]['AvgPointsPerGame']*1.5)+
        (x19*Download.iloc[9]['AvgPointsPerGame'])+(x20*Download.iloc[9]['AvgPointsPerGame']*1.5)+
        (x21*Download.iloc[10]['AvgPointsPerGame'])+(x22*Download.iloc[10]['AvgPointsPerGame']*1.5)+
        (x23*Download.iloc[11]['AvgPointsPerGame'])+(x24*Download.iloc[11]['AvgPointsPerGame']*1.5)+
        (x25*Download.iloc[12]['AvgPointsPerGame'])+(x26*Download.iloc[12]['AvgPointsPerGame']*1.5)+
        (x27*Download.iloc[13]['AvgPointsPerGame'])+(x28*Download.iloc[13]['AvgPointsPerGame']*1.5)+
        (x29*Download.iloc[14]['AvgPointsPerGame'])+(x30*Download.iloc[14]['AvgPointsPerGame']*1.5)+
        (x31*Download.iloc[15]['AvgPointsPerGame'])+(x32*Download.iloc[15]['AvgPointsPerGame']*1.5)+
        (x33*Download.iloc[16]['AvgPointsPerGame'])+(x34*Download.iloc[16]['AvgPointsPerGame']*1.5)+
        (x35*Download.iloc[17]['AvgPointsPerGame'])+(x36*Download.iloc[17]['AvgPointsPerGame']*1.5)+
        (x37*Download.iloc[18]['AvgPointsPerGame'])+(x38*Download.iloc[18]['AvgPointsPerGame']*1.5)+
        (x39*Download.iloc[19]['AvgPointsPerGame'])+(x40*Download.iloc[19]['AvgPointsPerGame']*1.5)+
        (x41*Download.iloc[20]['AvgPointsPerGame'])+(x42*Download.iloc[20]['AvgPointsPerGame']*1.5)+    
        (x43*Download.iloc[21]['AvgPointsPerGame'])+(x44*Download.iloc[21]['AvgPointsPerGame']*1.5)+    
        (x45*Download.iloc[22]['AvgPointsPerGame'])+(x46*Download.iloc[22]['AvgPointsPerGame']*1.5)))
 
m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0]))
print('x2: ' + str(x2.value[0]))
print('x3: ' + str(x3.value[0]))
print('x4: ' + str(x4.value[0]))
print('x5: ' + str(x5.value[0]))
print('x6: ' + str(x6.value[0]))
print('x7: ' + str(x7.value[0]))
print('x8: ' + str(x8.value[0]))
print('x9: ' + str(x9.value[0]))
print('x10: ' + str(x10.value[0]))
print('x11: ' + str(x11.value[0]))
print('x12: ' + str(x12.value[0]))
print('x13: ' + str(x13.value[0]))
print('x14: ' + str(x14.value[0]))
print('x15: ' + str(x15.value[0]))
print('x16: ' + str(x16.value[0]))
print('x17: ' + str(x17.value[0]))
print('x18: ' + str(x18.value[0]))
print('x19: ' + str(x19.value[0]))
print('x20: ' + str(x20.value[0]))
print('x21: ' + str(x21.value[0]))
print('x22: ' + str(x22.value[0]))
print('x23: ' + str(x23.value[0]))
print('x24: ' + str(x24.value[0]))
print('x25: ' + str(x25.value[0]))
print('x26: ' + str(x26.value[0]))
print('x27: ' + str(x27.value[0]))
print('x28: ' + str(x28.value[0]))
print('x29: ' + str(x29.value[0]))
print('x30: ' + str(x30.value[0]))
print('x31: ' + str(x31.value[0]))
print('x32: ' + str(x32.value[0]))
print('x33: ' + str(x33.value[0]))
print('x34: ' + str(x34.value[0]))
print('x35: ' + str(x35.value[0]))
print('x36: ' + str(x36.value[0]))
print('x37: ' + str(x37.value[0]))
print('x38: ' + str(x38.value[0]))
print('x39: ' + str(x39.value[0]))
print('x40: ' + str(x40.value[0]))
print('x41: ' + str(x41.value[0]))
print('x42: ' + str(x42.value[0]))
print('x43: ' + str(x43.value[0]))
print('x44: ' + str(x44.value[0]))
print('x45: ' + str(x45.value[0]))
print('x46: ' + str(x46.value[0]))

apm 108.183.51.97_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           70
   Intermediates:            0
   Connections  :            0
   Equations    :           29
   Residuals    :           29
 
 Number of state variables:             70
 Number of total equations: -           28
 Number of slack variables: -           24
 ---------------------------------------
 Degrees of freedom       :             18
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.27E+02 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi